# Collision Avoidance - Train Model

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Upload and extract dataset

Before you start, you should upload the ``dataset.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot.

You should then extract this dataset by calling the command below

In [3]:
!unzip -q dataset.zip

replace dataset/free/xy_065_111_5e93704e-2bf0-11f0-a827-e0e1a91dc0f8.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
!ls dataset2
!ls dataset2/other
!ls dataset2/shoes


other  shoes
01e55d38-3601-11f0-8da6-be7abd730879.jpg
01e609e0-3601-11f0-8da6-be7abd730879.jpg
030ddcca-35fd-11f0-a410-be7abd730879.jpg
03d990f4-35fd-11f0-a410-be7abd730879.jpg
05f43124-3601-11f0-8da6-be7abd730879.jpg
05f4b478-3601-11f0-8da6-be7abd730879.jpg
0607b892-35fd-11f0-a410-be7abd730879.jpg
0704175e-35fd-11f0-a410-be7abd730879.jpg
09302a04-35fd-11f0-a410-be7abd730879.jpg
0ac2dfd8-35fd-11f0-a410-be7abd730879.jpg
0c4cacc2-3601-11f0-8da6-be7abd730879.jpg
0c4d2b0c-3601-11f0-8da6-be7abd730879.jpg
14b9485c-3601-11f0-8da6-be7abd730879.jpg
14b9e672-3601-11f0-8da6-be7abd730879.jpg
1603d8a2-35fd-11f0-a410-be7abd730879.jpg
16ec442a-35fd-11f0-a410-be7abd730879.jpg
18f3a054-3604-11f0-8a5a-e0e1a91dc0f8.jpg
20d0a21a-35fd-11f0-a410-be7abd730879.jpg
21ed010a-3604-11f0-8a5a-e0e1a91dc0f8.jpg
242d69a0-3604-11f0-8a5a-e0e1a91dc0f8.jpg
264ada80-35fd-11f0-a410-be7abd730879.jpg
26ccb7d8-3604-11f0-8a5a-e0e1a91dc0f8.jpg
2a3749a6-3604-11f0-8a5a-e0e1a91dc0f8.jpg
2c0ede24-3604-11f0-8a5a-e0e1a91dc0f8.jpg
2e7

In [3]:
!find dataset2 -type d -name ".ipynb_checkpoints" -exec rm -rf {} +

In [4]:
from torchvision.datasets import ImageFolder
from PIL import UnidentifiedImageError

class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
            if self.transform is not None:
                sample = self.transform(sample)
            return sample, target
        except UnidentifiedImageError:
            print(f"Skipping corrupt image: {path}")
            return self.__getitem__((index + 1) % len(self))  # Retry with next image


You should see a folder named ``dataset`` appear in the file browser.

### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [6]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset2 = SafeImageFolder('dataset2', transform=transform)


### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [7]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset2, [len(dataset2) - 50, 50])

### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [8]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### Define the neural network

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

In [9]:
model = models.alexnet(pretrained=True)

The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has two class labels!  We'll replace
the final layer with a new, untrained layer that has only two outputs.  

In [10]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [11]:
print(2)

2


Finally, we transfer our model for execution on the GPU

In [12]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [13]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model2.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Skipping corrupt image: dataset2/other/4bf5014c-3603-11f0-ac94-e0e1a91dc0f8.jpg
Skipping corrupt image: dataset2/other/45089704-3603-11f0-ac94-e0e1a91dc0f8.jpg
Skipping corrupt image: dataset2/other/48f04aa6-3603-11f0-ac94-e0e1a91dc0f8.jpg
Skipping corrupt image: dataset2/other/0c4d2b0c-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/14b9485c-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/14b9e672-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/05f43124-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/05f4b478-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/6f394ca4-3602-11f0-bdaf-e0e1a91dc0f8.jpg
Skipping corrupt image: dataset2/other/14b9e672-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/0c4cacc2-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/0c4d2b0c-3601-11f0-8da6-be7abd730879.jpg
Skipping corrupt image: dataset2/other/1

KeyboardInterrupt: 

Once that is finished, you should see a file ``best_model.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation